In [52]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [53]:
tf.__version__

'2.4.1'

In [54]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [55]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


In [56]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [58]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [59]:
np.shape(X_train)

(8000, 12)

In [60]:
ann = tf.keras.models.Sequential()

In [61]:
ann.add(tf.keras.layers.Dense(units=6, activation='softsign'))

In [62]:
ann.add(tf.keras.layers.Dense(units=6, activation='softsign'))

In [63]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [64]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [65]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6248 - accuracy: 0.6615
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4709 - accuracy: 0.8047
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4369 - accuracy: 0.8076
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4339 - accuracy: 0.8073
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4294 - accuracy: 0.8113
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4230 - accuracy: 0.8176
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4172 - accuracy: 0.8172
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4134 - accuracy: 0.8204
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3973 - accuracy: 0.8303
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4187 - accura

In [66]:
y_pred = ann.predict(X_test)
y_pred

array([[0.21648073],
       [0.18575627],
       [0.11178491],
       ...,
       [0.10266495],
       [0.10961354],
       [0.29653308]], dtype=float32)

In [67]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [68]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1521   74]
 [ 193  212]]


A aluna Adne Moretti me ajudou na realização dessa atividade.

In [69]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

In [70]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [71]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [72]:
from keras.layers import Dropout

In [73]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [74]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [75]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

In [76]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [80]:
classifier = KerasClassifier(build_fn = build_classifier)

parameters = {'batch_size': [1, 2, 3],
              'epochs': [1, 8],
              'optimizer': ['adam', 'rmsprop']}

In [81]:
grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [82]:
grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

7200/7200 [==============================] - 7s 977us/step - loss: 0.4915 - accuracy: 0.7962


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


7200/7200 [==============================] - 7s 973us/step - loss: 0.5098 - accuracy: 0.7867
Epoch 1/8
7200/7200 [==============================] - 7s 942us/step - loss: 0.4783 - accuracy: 0.8060
Epoch 2/8
7200/7200 [==============================] - 7s 958us/step - loss: 0.4011 - accuracy: 0.8333
Epoch 3/8
7200/7200 [==============================] - 7s 1ms/step - loss: 0.4087 - accuracy: 0.8228
Epoch 4/8
7200/7200 [==============================] - 7s 956us/step - loss: 0.3981 - accuracy: 0.8307
Epoch 5/8
7200/7200 [==============================] - 7s 964us/step - loss: 0.4044 - accuracy: 0.8326
Epoch 6/8
7200/7200 [==============================] - 7s 959us/step - loss: 0.3874 - accuracy: 0.8395
Epoch 7/8
7200/7200 [==============================] - 7s 970us/step - loss: 0.3904 - accuracy: 0.8398
Epoch 8/8
7200/7200 [==============================] - 7s 1ms/step - loss: 0.3989 - accuracy: 0.8295
Epoch 1/8
7200/7200 [==============================] - 7s 970us/step - loss: 0.4906 - a